In [ ]:
import os
import kagglehub
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

1. Download the dataset using kagglehub

In [ ]:
print("Downloading dataset... Please wait.")
path = kagglehub.dataset_download("mohamedeldakrory8/ecg-heart-categorization-dataset-image-version")
print("Path to dataset files:", path)

In [ ]:
content_folder_name = "ECG Heartbeat Categorization Dataset Image Version"
data_dir = os.path.join(path, content_folder_name)

# Define train and test paths
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

2. Path Validation and Auto-Correction

In [ ]:
if not os.path.exists(train_dir):
    print("Warning: Expected 'train' path not found. Checking directory structure...")
    print("Main directory contents:", os.listdir(path))

    # Attempt to find the correct sub-folder automatically
    sub_folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
    if sub_folders:
        data_dir = os.path.join(path, sub_folders[0])
        train_dir = os.path.join(data_dir, 'train')
        test_dir = os.path.join(data_dir, 'test')
        print(f"Corrected Data Directory: {data_dir}")
    else:
        print("Error: No sub-folders found in the downloaded path.")

3. ImageDataGenerator Setup
According to the article, images are resized to 224x224

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

4. Load Training Data

In [ ]:
print("\nLoading Training Set...")
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

5. Load Validation/Test Data

In [ ]:
print("Loading Test Set...")
val_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Final Dataset Summary

In [ ]:
print("-" * 30)
print(f"Number of classes found: {train_data.num_classes}")
print(f"Class labels (Disease types): {list(train_data.class_indices.keys())}")
print("-" * 30)

Define the Hybrid DLA-ANN Model Architecture

In [ ]:
def build_final_model(num_classes=7):
    base_model = tf.keras.applications.VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.1),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(256),
        layers.Activation('swish'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Recall(name='sensitivity'), tf.keras.metrics.Precision(name='precision')]
    )
    return model

model = build_final_model(num_classes=7)
print("DLA-ANN Model Summary:")
model.summary()

BATCH_SIZE = 128
train_data.batch_size = BATCH_SIZE
val_data.batch_size = BATCH_SIZE

In [ ]:
print("\nStarting Optimized Training Process...")
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    steps_per_epoch=100,
    validation_steps=50
)

In [ ]:
model.save('ecg_dla_ann_model.h5')
print("\n" + "="*50)
print("Model saved successfully as 'ecg_dla_ann_model.h5'")
print("Based on Hybrid deep learning framework for heart disease prediction.")
print("="*50)

In [ ]:
print("\n" + "="*30)
print("Final Model Evaluation on Test Data")
print("="*30)
results = model.evaluate(val_data, steps=len(val_data))
metrics_names = model.metrics_names
for name, value in zip(metrics_names, results):
    if name != 'loss':
        print(f"Final {name.capitalize()}: {value*100:.2f}%")
    else:
        print(f"Final Loss: {value:.4f}")
print("\nComparison with Article 3 Targets:")
print(f"Target Accuracy: 93.6% | Your Accuracy: {results[1]*100:.2f}%")
print(f"Target Sensitivity: 97.4% | Your Sensitivity: {results[2]*100:.2f}%")

In [ ]:
model_path = 'ecg_dla_ann_model.h5'
if os.path.exists(model_path):
    inference_model = load_model(model_path)
    print("Model loaded successfully.")
else:
    print("Error: Saved model file not found!")

In [ ]:
random_idx = np.random.randint(0, len(val_data.filepaths))
img_path = val_data.filepaths[random_idx]
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_batch = np.expand_dims(img_array, axis=0)

In [ ]:
img = image.load_img(img_path, target_size=(224, 224))
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.title("Selected ECG Signal Image")
plt.axis('off')
plt.show()

In [ ]:
loaded_model = load_model('ecg_dla_ann_model.h5')
preds = loaded_model.predict(img_batch)
class_idx = np.argmax(preds)
label_code = list(train_data.class_indices.keys())[class_idx]
prob = preds[0][class_idx] * 100

موارد زیر تنها برای نمایش خروجی فارسی به کاربر می باشد

In [ ]:
fa_labels = {
    'N': {'title': 'وضعیت سالم (Normal)', 'msg': 'قلب شما به طور طبیعی می‌تپد. مشکلی مشاهده نشد.', 'color': 'green'},
    'M': {'title': 'هشدار بحرانی (Myocardial Infarction)', 'msg': 'علائم سکته قلبی تشخیص داده شد. فوراً به پزشک مراجعه کنید!', 'color': 'red'},
    'S': {'title': 'ناهنجاری دهلیزی (S)', 'msg': 'ضربان نامنظم در بخش بالایی قلب تشخیص داده شد. با پزشک مشورت کنید.', 'color': 'orange'},
    'V': {'title': 'ناهنجاری بطنی (V)', 'msg': 'ضربان نامنظم در بخش پایینی قلب تشخیص داده شد. بررسی پزشکی توصیه می‌شود.', 'color': 'orange'},
    'F': {'title': 'ضربان ترکیبی (F)', 'msg': 'اختلال جزئی در ریتم تپش مشاهده شد.', 'color': 'orange'},
    'F-resample': {'title': 'اختلال ریتم (F-res)', 'msg': 'ناهنجاری در ریتم ضربان قلب شناسایی شد.', 'color': 'orange'},
    'Q': {'title': 'نامشخص (Unknown)', 'msg': 'کیفیت تصویر برای تشخیص پایین است یا سیگنال نامفهوم است.', 'color': 'gray'}
}
predictions = inference_model.predict(img_batch)
predicted_class_index = np.argmax(predictions)
info = fa_labels.get(label_code, {'title': 'خطا', 'msg': 'دسته ناشناخته', 'color': 'black'})

In [ ]:
class_labels = list(train_data.class_indices.keys())
predicted_label = class_labels[predicted_class_index]
confidence = predictions[0][predicted_class_index] * 100
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.title(f"Predicted: {predicted_label} ({confidence:.2f}%)", color='green')
plt.axis('off')
plt.show()

In [ ]:
print(f"--- گزارش تحلیل قلب ---")
print(f"نتیجه تشخیص: {info['title']}")
print(f"توضیح برای کاربر: {info['msg']}")
print(f"میزان اطمینان سیستم: {prob:.2f} درصد")